In [41]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

Recuperar base já pré-processada

In [43]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_2023_full.pkl')

variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
# Apenas utilizado nos modelos fianais, BayesSearchCV não utiliza Eval Set já possui validacao cruzada interna
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

### 1. Modelo base

In [44]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train_final, 
                y_train_final['NUM_NOTA_CH'], 
                eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4296
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 40
[LightGBM] [Info] Start training from score 527.881246


In [45]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [8]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base',
                 descricao_modelo='Modelo LGBMRegressor base')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_base' already exists. Creating a new version of this model...
2025/06/07 23:02:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation.

🏃 View run resilient-pig-154 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/95fb16e000d14a73be8a3e660aa993f5
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_base
Rastreamento do MLflow finalizado.


In [46]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 53.8265
RMSE (treino): 68.2660
R2 (treino): 0.3498
MAE (teste): 55.2285
RMSE (teste): 70.0224
R2 (teste): 0.3122


In [47]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_lgbm, 'Modelos\modelo_lgbm_base.pkl')

['Modelos\\modelo_lgbm_base.pkl']

### 2. Bayes Search

In [48]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [49]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (5, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (40, 100),                        # Profundidade máxima da árvore
    'learning_rate': (0.005, 0.1, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                  # Número de árvores
    'subsample': (0.3, 1.0),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.2, 1.0),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1.0, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-5, 1.0, 'log-uniform'),      # Regularização L2
}

In [50]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=30,                      # Número de avaliações do modelo
    cv=5,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [51]:
fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],               # Métricas a serem avaliadas
    'categorical_feature': categorical_features,        # Colunas categóricas
}

In [53]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train, y_train['NUM_NOTA_CH'], **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [ ]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {
        "colsample_bytree": 0.2297823151086026,
        "learning_rate": 0.028382425255806625,
        "max_depth": 40,
        "n_estimators": 5990,
        "num_leaves": 40,
        "reg_alpha": 0.00596084139054512,
        "reg_lambda": 0.8601375912019539,
        "subsample": 0.9434176158805601
    }
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Erro ao obter melhores parâmetros, usando valores calculados anteriormente:
 {'colsample_bytree': 0.2297823151086026, 'learning_rate': 0.028382425255806625, 'max_depth': 40, 'n_estimators': 5990, 'num_leaves': 40, 'reg_alpha': 0.00596084139054512, 'reg_lambda': 0.8601375912019539, 'subsample': 0.9434176158805601}


In [ ]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_final,
                      y_train_final['NUM_NOTA_CH'],
                      eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4296
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 40
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2901]	valid_0's rmse: 69.8154	valid_0's l1: 55.0559	valid_0's l2: 4874.18


In [ ]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [21]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_bayes' already exists. Creating a new version of this model...
2025/06/07 23:26:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation

🏃 View run amazing-carp-611 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/40540fd080d74b25b19b1833aee7ca39
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes
Rastreamento do MLflow finalizado.


Created version '5' of model 'modelo_lgbm_bayes'.


In [ ]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 53.3724
RMSE (treino): 67.7531
R2 (treino): 0.3595
MAE (teste): 54.8517
RMSE (teste): 69.5887
R2 (teste): 0.3207


In [ ]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_lgbm_bayes, 'Modelos\modelo_lgbm_bayes.pkl')

['Modelos\\modelo_lgbm_bayes.pkl']

### 3. Buscar melhorar modelo alterando Features

#### a. Tentar melhorar removendo todas as variáveis sem categorias definidas ("Não informado")

    TP_ESTADO_CIVIL: 0 (Não informado)
    TP_COR_RACA: 0 (Não declarado)
    TP_NACIONALIDADE: 0 (Não informado)
    TP_ESCOLA: 1 (Não Respondeu)
    TP_ENSINO: 0 (Não informado)

In [ ]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_2023_full.pkl')

# Remover linhas com valores não explicativos
df_enem = df_enem[
    (df_enem['CAT_NACIONALIDADE'] != 0) &
    (df_enem['CAT_ENSINO'] != 0) &
    (df_enem['CAT_COR_RACA'] != 0) &
    (df_enem['CAT_ESTADO_CIVIL'] != 0) &
    (df_enem['CAT_ESCOLA'] != 1)
]

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

In [ ]:
# Criar o modelo LGBMRegressor sem categorias definidas
modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas = lgbm.LGBMRegressor(random_state=42,
                                                                                max_bin=4095, 
                                                                                force_row_wise=True)

# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.fit(X_train_final, 
                                                                y_train_final['NUM_NOTA_CH'], 
                                                                eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                                                                eval_metric=['r2', 'rmse', 'mae'],
                                                                categorical_feature=categorical_features,
                                                                callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4237
[LightGBM] [Info] Number of data points in the train set: 457203, number of used features: 40
[LightGBM] [Info] Start training from score 529.413394
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[3118]	valid_0's rmse: 69.7432	valid_0's l1: 54.9119	valid_0's l2: 4864.11


In [27]:
# Previsões
y_pred_explicativas = modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.predict(X_test)

In [ ]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas,
                    parametros={**modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_explicativas,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV sem categorias definidas')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas' already exists. Creating a new version of this model...
2025/06/07 23:37:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seco

🏃 View run suave-shark-495 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/303d650dedda447b920aaa527264eb0e
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas
Rastreamento do MLflow finalizado.


Created version '3' of model 'modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas'.


In [28]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_explicativas, "teste")

MAE (treino): 52.8961
RMSE (treino): 67.2000
R2 (treino): 0.3653
MAE (teste): 54.6748
RMSE (teste): 69.5368
R2 (teste): 0.3220


### 2. Tentar melhorar removendo variáveis de menor importância (importância e informação mútua)

    CAT_SIT_FUNC_ESC
    CAT_NACIONALIDADE
    CAT_ENSINO
    CAT_LOCALIZACAO_ESC
    CAT_ESTADO_CIVIL
    BIN_Q002_DUMMY_H
    NUM_Q017
    NUM_Q012
    BIN_Q001_DUMMY_H
    NUM_Q015

In [ ]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_2023_full.pkl')

# Remover algumas colunas com MENOR valor explicativo
df_enem = df_enem.drop(columns=['CAT_SIT_FUNC_ESC',
                                'CAT_NACIONALIDADE',
                                'CAT_ENSINO',
                                'CAT_LOCALIZACAO_ESC',
                                'CAT_ESTADO_CIVIL',
                                'BIN_Q002_DUMMY_H',
                                'NUM_Q017',
                                'NUM_Q012',
                                'BIN_Q001_DUMMY_H',
                                'NUM_Q015',
                                ])

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

In [ ]:
# Criar o modelo LGBMRegressor com redução de features
modelo_lgbm_reducao_de_features = lgbm.LGBMRegressor(random_state=42,
                                                        max_bin=4095, 
                                                        force_row_wise=True)

# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_reducao_de_features.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_reducao_de_features.fit(X_train_final,
                                    y_train_final['NUM_NOTA_CH'], 
                                    eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                                    eval_metric=['r2', 'rmse', 'mae'],
                                    categorical_feature=categorical_features,
                                    callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4258
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 30
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1291]	valid_0's rmse: 70.0128	valid_0's l1: 55.2252	valid_0's l2: 4901.79


In [32]:
# Previsões
y_pred_var_importantes = modelo_lgbm_reducao_de_features.predict(X_test)

In [ ]:
nome_experimento = 'Notas CH ENEM 2023'

# Avaliar o modelo
registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_reducao_de_features,
                    parametros={**modelo_lgbm_reducao_de_features.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_var_importantes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_reducao_de_10_features',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV com redução de 10 features')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_reducao_de_10_features' already exists. Creating a new version of this model...
2025/06/07 23:42:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version t

🏃 View run amazing-midge-269 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/3dcfbf3840ba41ceb594d915223aba03
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_reducao_de_10_features
Rastreamento do MLflow finalizado.


Created version '2' of model 'modelo_lgbm_reducao_de_10_features'.


In [33]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_reducao_de_features.predict(X_train), "treino")
# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_var_importantes, "teste")

MAE (treino): 53.9122
RMSE (treino): 68.3946
R2 (treino): 0.3469
MAE (teste): 55.0416
RMSE (teste): 69.7828
R2 (teste): 0.3169


### 3. Aplicar os dois: remover categorias não explicativas + remover colunas menos importantes

In [ ]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_2023_full.pkl')

# Remover linhas com valores não explicativos
df_enem = df_enem[
    (df_enem['CAT_NACIONALIDADE'] != 0) &
    (df_enem['CAT_ENSINO'] != 0) &
    (df_enem['CAT_COR_RACA'] != 0) &
    (df_enem['CAT_ESTADO_CIVIL'] != 0) &
    (df_enem['CAT_ESCOLA'] != 1)
]

# Remover algumas colunas com MENOR valor explicativo
df_enem = df_enem.drop(columns=['CAT_SIT_FUNC_ESC',
                                'CAT_NACIONALIDADE',
                                'CAT_ENSINO',
                                'CAT_LOCALIZACAO_ESC',
                                'CAT_ESTADO_CIVIL',
                                'BIN_Q002_DUMMY_H',
                                'NUM_Q017',
                                'NUM_Q012',
                                'BIN_Q001_DUMMY_H',
                                'NUM_Q015',
                                ])

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

In [ ]:
# Criar o modelo LGBMRegressor com redução de features e categorias menos explicativas
modelo_lgbm_reducao_de_features_categorias_nao_explicativas = lgbm.LGBMRegressor(random_state=42,
                                                                                    max_bin=4095, 
                                                                                    force_row_wise=True)

# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_reducao_de_features_categorias_nao_explicativas.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_reducao_de_features_categorias_nao_explicativas.fit(X_train_final,
                                                                y_train_final['NUM_NOTA_CH'], 
                                                                eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                                                                eval_metric=['r2', 'rmse', 'mae'],
                                                                categorical_feature=categorical_features,
                                                                callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4201
[LightGBM] [Info] Number of data points in the train set: 457203, number of used features: 30
[LightGBM] [Info] Start training from score 529.413394
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2577]	valid_0's rmse: 69.9135	valid_0's l1: 55.0636	valid_0's l2: 4887.9


In [39]:
# Previsões
y_pred_var_feat_categ = modelo_lgbm_reducao_de_features_categorias_nao_explicativas.predict(X_test)

In [ ]:
nome_experimento = 'Notas CH ENEM 2023'

# Avaliar o modelo
registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_reducao_de_features_categorias_nao_explicativas,
                    parametros={**modelo_lgbm_reducao_de_features_categorias_nao_explicativas.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=modelo_lgbm_reducao_de_features_categorias_nao_explicativas.predict(X_test),
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_reducao_de_10_features_categorias_nao_explicativas',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV com redução de 10 features e categorias não explicativas')

In [40]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_reducao_de_features_categorias_nao_explicativas.predict(X_train), "treino")
# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_var_feat_categ, "teste")

MAE (treino): 53.3024
RMSE (treino): 67.6805
R2 (treino): 0.3562
MAE (teste): 54.7989
RMSE (teste): 69.6746
R2 (teste): 0.3193


Conclusão

- Remoção de categorias não explicativas
    - mantendo fixo os parâmetros do modelo e removendo apenas as linhas com categorias não explicativas houve uma melhora do modelo
    - R² de 0.3207 para 0.3220

- Remoção das 10 variáveis menos explicativas
    - mantendo fixo os parâmetros do modelo e removendo apenas as colunas com menor importância para o alvo houve uma piora do modelo
    - Removendo 10: R² de 0.3207 para 0.3169

- Aplicando as duas ténicas
    - Redução R² de 0.3207 para 0.3193

Padrão se mantém para outras métricas RMSE e MAE

É interessante remover todas as categorias não explicativas, mostra uma melhora no modelo, mas manter todas as variáveis disponíveis, mesmo se menos importantes.